## Librerias y configuracion inicial.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

df = pd.read_csv("spotify_dataset.csv")
print(" Dataset shape:", df.shape)
print("\n Data types:\n", df.dtypes)
print("\n Missing values:\n", df.isnull().sum())
print("\n Duplicate rows:", df.duplicated().sum())
df.head()

###  Estructura General del Dataset

El dataset contiene **114,000 filas** y **21 columnas**, lo que representa una muestra amplia y rica para el análisis musical.

#### Tipos de Datos:
- Las columnas se dividen principalmente entre:
  - **Variables numéricas continuas** (`float64`), como `danceability`, `energy`, `tempo`, etc.
  - **Variables categóricas o de texto** (`object`), como `track_id`, `artists`, `track_name`, `track_genre`.
  - **Variables discretas enteras** (`int64`), como `popularity`, `key`, `mode`, `time_signature`.
  - Una variable booleana (`explicit`) que indica si la canción tiene contenido explícito.

Este esquema de datos es adecuado tanto para análisis estadístico clásico como para algoritmos de machine learning supervisado y no supervisado.

#### Revisión de Calidad Inicial:  (Sujeto a una segunda revisión de esta hipotesis al final del EDA)
-  **No se encontraron valores nulos ni duplicados**, lo que indica que el dataset ya ha sido preprocesado adecuadamente o curado de forma controlada.
- La columna `Unnamed: 0` es un **índice generado automáticamente** durante la exportación. Puede eliminarse si no se requiere para trazabilidad o identificación.

En general, la estructura de este dataset es robusta, bien definida y lista para exploración profunda y modelamiento.


In [ ]:
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
desc_stats = df[numerical_cols].describe().T
desc_stats["missing"] = df[numerical_cols].isnull().sum()
desc_stats["skew"] = df[numerical_cols].skew()
desc_stats["kurtosis"] = df[numerical_cols].kurt()
desc_stats

###  Estadísticas Descriptivas, Asimetría y Curtosis

El resumen estadístico permite identificar tendencias, dispersión y posibles anomalías en las variables numéricas:

####  `popularity`
- Media = 33.2, mediana = 35: distribución aproximadamente **simétrica**.
- Rango completo desde 0 hasta 100.
- Ligera asimetría positiva (skew = 0.04), curtosis ligeramente negativa.

####  `duration_ms`
- Media ≈ 3.8 minutos (228,029 ms).
- Valor **máximo extremadamente alto**: 5,237,295 ms (~87 minutos) — probable outlier.
- **Alta asimetría (11.2)** y **curtosis extrema (354.9)**: distribución altamente sesgada.

####  `danceability`, `energy`, `valence`
- Distribuciones razonablemente normales.
- `danceability` con leve sesgo negativo.
- `valence` (emocionalidad positiva) está centrada en 0.47, lo cual sugiere un balance entre temas felices y neutros.

####  `loudness`
- Rango de -49 dB a +4.5 dB (fuertemente asimétrica a la izquierda).
- **Curtosis negativa** (menos extremos que una distribución normal), pero aún así valores extremos como -49.5 dB indican outliers.

####  `speechiness`, `instrumentalness`, `acousticness`
- Todas muestran **alta asimetría positiva**, indicando que la mayoría de canciones tienen valores cercanos a 0.
- `speechiness` tiene curtosis de **28.8**, lo que indica presencia de valores extremos (posiblemente canciones 100% habladas como podcasts o spoken word).
- `instrumentalness` y `acousticness` tienen distribución polarizada: muchas canciones cantadas o electrónicas, pero también presencia significativa de instrumentales o acústicas puras.

####  `tempo`
- Rango razonable (hasta 243 BPM), media ≈ 122 BPM.
- Asimetría moderada, distribución saludable para análisis rítmico.

####  `time_signature`
- La mayoría tiene compás de **4/4** (mediana = 4.0), lo cual es estándar en música pop.
- Asimetría y curtosis elevadas debido a los pocos valores diferentes (e.g., 3/4, 5/4).

###  Conclusiones Clave:
- Las variables `duration_ms`, `speechiness`, `instrumentalness` y `acousticness` muestran **sesgo fuerte o valores extremos**, por lo que requieren especial atención en visualización, normalización o eliminación de outliers.
- La mayoría de las variables están distribuidas adecuadamente para modelamiento predictivo, con algunas transformaciones opcionales según el objetivo.


In [ ]:
print("Top 10 artistas con más canciones:")
print(df['artists'].value_counts().head(10))

print("\nTop 10 géneros musicales:")
print(df['track_genre'].value_counts().head(10))

###  Top 10 Artistas con Más Canciones

El dataset está dominado por artistas legendarios como **The Beatles** (279 canciones), **George Jones** y **Stevie Wonder**, lo cual sugiere una fuerte presencia de música clásica y de catálogo. Esto puede influir en las métricas generales como duración y popularidad.

###  Top 10 Géneros Musicales

Todos los géneros más frecuentes tienen exactamente **1000 canciones**, lo que indica que el dataset ha sido construido de forma **balanceada por género**. Esto es ideal para análisis comparativos y modelos de clasificación, ya que se evita el sesgo por clase desbalanceada.


In [ ]:
sns.histplot(df['popularity'], bins=30, kde=True, color='green')
plt.title("Distribución de Popularidad de Canciones")
plt.xlabel("Popularidad (0 a 100)")
plt.ylabel("Frecuencia")
plt.show()

###  Distribución de Popularidad

La popularidad de las canciones está **sesgada hacia la izquierda**, con un pico muy alto en el rango 0–10. Esto implica que la mayoría de las canciones tienen baja popularidad en plataformas como Spotify.

No obstante, también se observa una distribución bimodal secundaria, con picos adicionales en los rangos 20–40 y 50–60. Esto podría deberse a canciones con cierto reconocimiento pero que no alcanzan los niveles virales.

Este sesgo debe tenerse en cuenta al entrenar modelos de predicción, y puede sugerir estrategias de marketing digital enfocadas en este segmento intermedio.


In [ ]:
df['duration_min'] = df['duration_ms'] / 60000
sns.kdeplot(df['duration_min'], fill=True, label="Duración (min)")
sns.kdeplot(df['tempo'], fill=True, label="Tempo (BPM)")
plt.title("Distribución de Duración y Tempo")
plt.legend()
plt.show()

###  Distribución de Duración y Tempo

La duración de las canciones tiene una distribución normal sesgada hacia la izquierda, con la mayoría de los temas entre **2.5 y 4 minutos**, como es típico en la industria musical comercial.

En cambio, el **tempo (BPM)** muestra múltiples picos alrededor de **100 a 130 BPM**, indicando diferentes estilos rítmicos comunes (pop, electrónica, reguetón). Algunos valores extremos por encima de 200 BPM podrían representar outliers o errores de carga.

Este análisis ayuda a identificar las "zonas de confort" de los géneros más consumidos.


In [ ]:
for feature in ['speechiness', 'acousticness', 'instrumentalness']:
    sns.histplot(df[feature], kde=True, bins=30)
    plt.title(f"Distribución de: {feature}")
    plt.show()

###  Distribución de Speechiness

La mayoría de las canciones tienen valores de speechiness cercanos a **0.05**, lo que sugiere que predominan las canciones **cantadas** y no habladas.

Valores altos (cercanos a 1.0) son raros, pero pueden representar **podcasts, spoken word o rap explícito**. Este tipo de análisis permite detectar automáticamente contenido hablado o mixto.

Esta variable es útil para segmentar entre géneros líricos y géneros hablados.


###  Distribución de Acousticness

La mayoría de las canciones tienen un valor de acousticness cercano a **0**, lo cual indica que predomina la producción digital o electrónica sobre los instrumentos acústicos puros.

Sin embargo, también hay un **ligero repunte cerca del valor 1**, lo que sugiere una presencia significativa —aunque mucho menor— de canciones acústicas puras (guitarra, piano, voz sin procesamiento).

Esta distribución fuertemente sesgada a la izquierda es común en datasets con diversidad de géneros modernos y sugiere que la música acústica representa un **nicho de mercado específico**.


###  Distribución de Instrumentalness

La gran mayoría de las canciones tienen un valor de instrumentalness inferior a **0.1**, lo que indica que son canciones con voz o canto, y por tanto no instrumentales.

Un pequeño pico hacia el valor **1.0** evidencia la presencia de pistas completamente instrumentales (música ambiental, clásica, electrónica sin vocales). Este grupo minoritario puede ser clave para tareas de clasificación musical o detección automática de contenido sin voz.

La fuerte asimetría hacia valores bajos implica que la voz sigue siendo **el componente central de la mayoría de las producciones musicales modernas**.


In [13]:
df.dropna(subset=['track_name', 'artists', 'album_name'], inplace=True)
df.drop_duplicates(inplace=True)

In [ ]:
audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
                  'instrumentalness', 'liveness', 'valence', 'tempo']

sns.heatmap(df[audio_features].corr(), annot=True, cmap='coolwarm')
plt.title("Matriz de Correlación entre Características de Audio")
plt.show()

###  Correlación entre Características de Audio

- **Energy y Loudness** tienen una **alta correlación positiva (0.76)**, lo cual es lógico, ya que las canciones más energéticas suelen estar producidas con mayor volumen percibido.
- **Acousticness** tiene correlaciones **negativas fuertes** con energy (-0.73) y loudness (-0.59), reafirmando que las canciones acústicas tienden a ser suaves y menos procesadas.
- **Danceability y Valence** muestran una correlación moderada (0.48), indicando que las canciones más bailables suelen tener una carga emocional más positiva.
- **Instrumentalness y Valence** tienen una correlación negativa (-0.32), lo que sugiere que las canciones instrumentales tienden a ser más neutras o melancólicas.

Estas correlaciones guían el diseño de modelos de recomendación musical y análisis de estilo.


In [ ]:
for feature in audio_features:
    sns.scatterplot(x=feature, y='popularity', data=df, alpha=0.3)
    plt.title(f"Popularidad vs {feature}")
    plt.show()

## Limpieza de los datos y carga en nuevo dataset como "spotify_dataset_clean.csv"

In [ ]:

# Cargar el dataset original
df = pd.read_csv("spotify_dataset.csv")

# =============================
# 1. Detección de valores nulos
# =============================
print(" Valores nulos por columna:")
print(df.isnull().sum())

# =============================
# 2. Detección de strings vacíos o solo espacios
# =============================
# Aplicamos solo a columnas de tipo 'object' (textuales)
text_cols = df.select_dtypes(include='object').columns

for col in text_cols:
    vacios = df[col].apply(lambda x: isinstance(x, str) and x.strip() == '').sum()
    if vacios > 0:
        print(f" Valores vacíos detectados en '{col}': {vacios}")

# =============================
# 3. Reemplazo de strings vacíos por NaN y limpieza
# =============================
df[text_cols] = df[text_cols].applymap(lambda x: None if isinstance(x, str) and x.strip() == '' else x)

# =============================
# 4. Eliminación de filas con valores nulos críticos
# =============================
# Puedes ajustar aquí qué columnas son esenciales
columnas_críticas = ['track_name', 'artists', 'album_name']
df_clean = df.dropna(subset=columnas_críticas)

# =============================
# 5. Eliminación de duplicados exactos
# =============================
duplicados_antes = df_clean.duplicated().sum()
print(f"🧹 Duplicados encontrados: {duplicados_antes}")
df_clean = df_clean.drop_duplicates()

# =============================
# 6. Guardar el nuevo dataset limpio
# =============================
df_clean.to_csv("spotify_dataset_clean.csv", index=False)
print(" Dataset limpio guardado como 'spotify_dataset_clean.csv'")


In [ ]:
# Mostrar forma y resumen rápido
print(" Shape del dataset limpio:", df_clean.shape)

# 1. Verificación general de nulos
print("\n Nulos por columna:")
print(df_clean.isnull().sum())

# 2. Confirmación total
if df_clean.isnull().sum().sum() == 0:
    print("\n Confirmado: No hay valores nulos en el dataset limpio.")
else:
    print("\n Aún hay valores nulos. Revisa las columnas con conteos mayores a 0.")

# 3. Ver muestra de datos para inspección visual
df_clean.head()